# TP1
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Problema 1

### Descrição do Problema

Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:

    1. Cada reunião ocupa uma sala (enumeradas 1...S) durante um “slot”  1..T (hora,dia).  
    
    2.  Cada reunião tem associado um projeto (enumerados 1..P) e um conjunto de participantes. Os diferentes colaboradores são enumerados 1..C.
    
    3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. 
    
    4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  50% do total de colaboradores do projeto. 

### Abordagem do Problema

Temos como input's do problema as variaveís $P$, $C$, $S$ e $T$, o conjunto de colaboradores, o líder e o número mínimo de reuniões semanais de cada projeto e disponibilidade de cada participante (representada em uma matriz 2x2).

Estamos perante um problema de alocação, e por isso, será necessário criar uma familia x_{p,c,s,t} de variáveis binárias, da seguinte maneira:

$$x_{p,c,s,t} = 1 \quad \mathbf{sse} \quad \mbox{$c$ tem reunião sobre o projeto $p$ na sala $s$ no slot $t$} $$

As variáveis $P\times C \times S \times T$ são apresentadas na matriz de alocalão $X$ com valores instânciados $\{0,1\}^{P\times C \times S \times T}$.

Também establelecemos algumas restrições para o solver:
- O lider participa em todas as reuniões sobre um determinado projeto;
- O projeto tem mínimo (“quorum”) de 50% do total de colaboradores do projeto;
- Se um colaborador não tem disponibilidade num certo slot não pode ter nenhuma reunião nesse slot;
- Todos os projetos devem ser alocados um número minimo de reuniões semanais;
- Colaboradores não podem participar em reuniões de projetos que não estão associados;
- Um colaborador não pode participar em duas reuniões ao mesmo tempo;
- Em cada sala não pode haver duas reuinões de dois projetos ao mesmo tempo;

Além disso, temos dois critérios de optimização:
- Maximizar o número de reuniões efetivamente realizadas;
- Minimizar o número médio de reuniões por participante.


## Código Python

Vamos utilizar a programação linear presente na biblioteca OR-Tools e criar o nosso solver com o nome de horario.

In [1]:
from ortools.linear_solver import pywraplp
horario = pywraplp.Solver.CreateSolver('SCIP')

### Inputs do Problema e Exemplos

Funções auxiliares:

In [2]:
def printmatrix(x,m,n):
    for i in range(1,m+1):
        for j in range(1,n+1):
            print(x[i,j], end = ' ')  
        print()

def creatematrix(x,string,m,n):
    l = 0
    for i in range(1,m+1):
        for j in range(1,n+1):
            x[i,j] = int(string[l])
            l += 1

def createdict(x,string,n):
    for i in range(n):
        x[i+1] = int(string[i])

Exemplo 1:

In [3]:
P,C,S,T = 6, 6, 12, 6

d = {}
creatematrix(d,"100101111101010110010111011101000100",C,T)
print("Matriz de disponibilidade: ")
printmatrix(d,C,T)
col = {}
creatematrix(col,"111111010110000001110100111101111110",P,C)
print("Matriz dos colaboradores: ")
printmatrix(col,P,C)
l = {}
print("Líderes: ")
createdict(l,"126435",P)
print(l)
print("Mínimo por projeto: ")
m = {1:1,2:1,3:1,4:1,5:0,6:1}
createdict(m,"111101",P)
print(m)

Matriz de disponibilidade: 
1 0 0 1 0 1 
1 1 1 1 0 1 
0 1 0 1 1 0 
0 1 0 1 1 1 
0 1 1 1 0 1 
0 0 0 1 0 0 
Matriz dos colaboradores: 
1 1 1 1 1 1 
0 1 0 1 1 0 
0 0 0 0 0 1 
1 1 0 1 0 0 
1 1 1 1 0 1 
1 1 1 1 1 0 
Líderes: 
{1: 1, 2: 2, 3: 6, 4: 4, 5: 3, 6: 5}
Mínimo por projeto: 
{1: 1, 2: 1, 3: 1, 4: 1, 5: 0, 6: 1}


Exemplo 2:

Exemplo 3:

Exemplo 4:

Exemplo 5:

Criação da matriz de alocação:

In [4]:
X = {}
for p in range(1,P+1):
    for c in range(1,C+1):
        for s in range(1,S+1):
            for t in range(1,T+1):
                X[p,c,s,t] = horario.BoolVar(f'X[{p}][{c}][{s}][{t}]')

### Apresentação das restrições

O lider participa em todas as reuniões sobre um determinado projeto:

$$
\forall_{p< P,c< C,s< S,t<T }  \quad x_{p,l_p,s,t} \ge x_{p,c,s,t}
$$

In [5]:
for p in range(1,P+1):
    for c in range(1,C+1):
        for s in range(1,S+1):
            for t in range(1,T+1):
                horario.Add(X[p,l[p],s,t] >= X[p,c,s,t])


O projeto tem mínimo (“quorum”) de 50% do total de colaboradores do projeto:
$$
\forall_{p< P,s< S,t< T} \quad \sum_{c< C} x_{p,c,s,t} \ge \frac{\sum_{c< C} col_{p,c}}{2}x_{p,l_p,s,t}
$$

In [6]:
for p in range(1,P+1):
    for s in range(1,S+1):
        for t in range(1,T+1):
            horario.Add(sum([ X[p,c,s,t] for c in range(1,C+1)]) >= sum([ col[p,c] for c in range(1,C+1) ])*X[p,l[p],s,t]/2)
                

Se um colaborador não tem disponibilidade num certo slot não pode ter nenhuma reunião nesse slot:
$$
\forall_{c< C,t< T} \quad d_{c,t} = 0 \quad \implies \quad \sum_{p<P,s< S} x_{p,c,s,t} = 0
$$

In [7]:
for c in range(1,C+1):
    for t in range(1,T+1):
        if d[c,t] == 0:
            horario.Add(sum([ X[p,c,s,t] for p in range(1,P+1) for s in range(1,S+1)]) == 0)

Todos os projetos devem ser alocados um número minimo de reuniões semanais:
$$
\forall_{p< P} \quad \sum_{s< S,t< T} x_{p,l_p,s,t} \ge m_p
$$

In [8]:
for p in range(1,P+1):
    horario.Add(sum([ X[p,l[p],s,t] for s in range(1,S+1) for t in range(1,T+1)]) >= m[p])

Colaboradores não podem participar em reuniões de projetos que não estão associados:
$$
\forall_{p< P,c< C,s< S,t< T} \quad x_{p,c,s,t} \le col_{p,c}
$$

In [9]:
for p in range(1,P+1):
    for c in range(1,C+1):
        for s in range(1,S+1):
            for t in range(1,T+1):
                horario.Add(X[p,c,s,t] <= col[p,c])

Um colaborador não pode participar em duas reuniões ao mesmo tempo:
$$
\forall_{c< C, t< T} \quad\sum_{p< P, s< S} x_{p,c,s,t} \le 1
$$

In [10]:
for c in range(1,C+1):
    for t in range(1,T+1):
        horario.Add(sum([ X[p,c,s,t] for p in range(1,P+1) for s in range(1,S+1)]) <= 1)

Em cada sala não pode haver duas reuinões de dois projetos ao mesmo tempo:
$$
\forall_{s< S, t< T} \quad \sum_{p< P} x_{p,l_p,s,t} \leq 1
$$

In [11]:
for s in range(1, S+1):
    for t in range(1,t+1):
        horario.Add(sum([ X[p,l[p],s,t] for p in range(1,P+1)]) <= 1)

### Apresentação das Otimizações

Primeiro critério de optimização: Maximizar o número de reuniões efetivamente realizadas
$$
\quad\sum_{p< P, s< S, t< T} x_{p,l_p,s,t}
$$

In [12]:
horario.Maximize(sum([X[p,l[p],s,t] for p in range(1,P+1) for s in range(1,S+1) for t in range(1,T+1)]))

Segundo critério de optimização: Minimizar o número médio de reuniões por participante
$$
\quad\sum_{p< P, c<C, s< S, t< T} x_{p,l_p,s,t}
$$

In [13]:
#horario.Minimize(sum(X[p,c,s,t] for p in range(1,P+1) for c in range(1,C+1) for s in range(1,S+1) for t in range(t, T+1)))

### Testes de aplicação dos exemplos

Exemplo 1:

In [14]:
status = horario.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print(1)
if status == pywraplp.Solver.INFEASIBLE:
    print(2)
if status == pywraplp.Solver.UNBOUNDED:
    print(3)

1


Exemplo 2:

In [15]:
status = horario.Solve()

Exemplo 3:

In [16]:
status = horario.Solve()

Exemplo 4:

In [17]:
status = horario.Solve()

Exemplo 5:

In [18]:
status = horario.Solve()